In [1]:
import boxes
import numpy as np
import os
import resource
import gc

In the below cell, we give every info needed to load the (max connected component) of the networks

Head tells the reader function how many row to skip that contain header lines

In [2]:
networks={'path':['../../nets/ca-CSphd','../../nets/bio-celegans','../../nets/socfb-Caltech36','../../nets/fulgidus',
            '../../nets/polbooks','../../nets/soc-dolphins','../../nets/bn-mouse_brain_1',
             '../../nets/road-minnesota','../../nets/ecoli','../../nets/tokyo'],
     'fname':['ca-CSphd.mtx','bio-celegans.mtx','socfb-Caltech36.mtx','Archaeoglobus-Fulgidus-Whole-Network.txt',
             'polbooks.mtx','soc-dolphins.mtx','bn-mouse_brain_1.edges','road-minnesota.mtx','ecoli.txt',
             'tokyo-metro.dot'],
     'name':['phd','cel','soc','ful','pol','dol','mou','min','eco','tok'],
     'head':[3,2,2,1,2,36,0,15,1,0]}

As it is apparent from the docs and tutorial, we will wrap up the whole boxing procedure in a loop.

Thus we will pass arguments to the appropriate algorithms as keyword arguments. To do this, we give these in a dictionary.


## Hyperparameters

Mostly cites from the original papers (see docs and paper)

### Differential evolution

'Our experiments show that DEBC algorithm converges
within a small number of generations. As shown in Fig.4(d),
we pick one length scale l B for four benchmark networks, __(CSF: lb=4, ecoli: lb=8, polbooks: lb=2, dolphins:lb=2 )__ the
results show that DEBC algorithm converges within the 70
generations. Due to the size of search space, the convergent
rate of DEBC algorithm is inverse proportion to l B , which
means the DEBC algorithm coverage faster with bigger l B'

Slowest convergence for CSF with:

'The Clustered scale-free network (CSF): The mod-
ified Barabási-Albert model with high clustering coefficient
[22]. Here, the applied CSF network has 2003 nodes and 8000
edges.
Since the CSF network has low modularity. We also applied
three clustered community networks (the polbooks, dolphin,
and football networks) as follows. These community networks
have both high modularity and clustering coefficient.'

'Nevertheless, all our results in this paper are reached within
5000 generation with population N P = 40. In our experi-
ments, we define the mutation control parameter F = 0.9 and
crossover probability CR = 0.85. And, we found that making
some adjustments of these two parameters have no obvious
influence on the final results.'

### Particle swarm optimization

'Here, we select random ω
ranging between [0, 1], and set c 1 and c 2 as the common used
value 1.494'

'In order to show the effectiveness of our PSOBC algorithm,
we apply 6 benchmark networks in our experiments (we
test the C.elegans network as both weighted and unweighted
network). Firstly, we show the box-covering results of PSOBC
algorithm on 3 unweighted networks and compare the results
with the greedy algorithm [14] and the Schneider algorithm
[17]. Secondly, we illustrate the box-covering results of the
PSOBC algorithm on 3 weighted networks and compare the
results with greedy algorithm. To verify the efficiency of our
algorithm, we also illustrate the result distribution P (N B )
of the box-covering algorithms. Moveover, we show the P-
SOBC algorithm has fast convergence rate. The parameters
of the PSOBC algorithm in this paper are set as: pop = 99,
G max < 1000'

'We apply the following benchmark networks:

(1) The Power Grid network (powerGrid): an undirected,
unweighted network representing the topology of the Western
States Power Grid of the United States [3].

(2) The Escherichia coli network (E.coli): a Protein-
protein Interaction network with 2859 proteins and 6890
interactions between them [29].

(3) and (4) The C.elegans network (C.elegans): a weight-
ed network representing the neural network of Caenorhabditis
elegans. Here, to show the differences between weighted and
unweighted network, we apply the box-covering algorithm on
C.elegans network as both weighted (p = −1 in Eq.11) and
unweighted network (p = 0 in Eq.11)

(5) The US air network (USAir97): direct flight connec-
tions between US airports in 1997. 
(6) The weighted fractal model (weiFracMod): a fractal
model that can produce weighted fractal network [30]. The
applied network has parameters as follows: initial nodes = 4,
branching rate = 3, iteration = 4 and p = 1'

Both small and relatively big N.

### Simulated annealing

'The typical starting temperature T is about
0.6 and the typical values of k 1 and k 2 are 5000 and 5,
respectively. Similar values are used by Zhou et al. in
their implementation of the SA algorithm [12]'

'We perform about 5000 steps at each
temperature, and then reduce T. The number of outer
cycles (temperature reductions) is k 3 , and it is set to
20, with cooling constant c set to 0.995 [12].'

'We used for the tests the software network related
to Java JDK 1.5 system, which includes the standard
Java libraries and development tools. The JDK
network has 8499 nodes and 42048 edges, so it can be
considered a quite large network.'

'We found that SA is the best algorithm in terms
of precision, but it is by far the worst in terms of
speed. The time performance of MA is better than GC
for large networks but the greedy coloring produces
more precise solutions. In conclusion, the Greedy
Coloring algorithm, based on the equivalence of the
box counting problem with the graph coloring
problem, looks the best compromise, having speed
comparable to MA, and accuracy comparable with
SA.'

### Sampling

Based on the figures, I gave a shot to n=10 and n=40.

In [3]:
algs_lb_dd={
    'cbb':{'alg':boxes.cbb,
           'kwargs':{'boxing':True}},
    'greedy':{'alg':boxes.greedy_coloring,
              'kwargs':{'boxing':True,'pso_position':False, 'strategy':'random_sequential'}},
    'obca':{'alg':boxes.obca, 
            'kwargs':{'boxing':True}},
    'de_70':{'alg':boxes.differential_evolution,
         'kwargs':{'num_p':40, 'big_f':0.9, 'cr':0.85, 'gn':70, 'boxing':True, 'dual_new':True}},
    'de_30':{'alg':boxes.differential_evolution,
         'kwargs':{'num_p':40, 'big_f':0.9, 'cr':0.85, 'gn':30, 'boxing':True, 'dual_new':True}},
    'pso_1000':{'alg':boxes.pso,
          'kwargs':{'gmax':1000, 'pop':99, 'c1':1.494, 'c2':1.494, 'boxing':True}},
    'pso_200':{'alg':boxes.pso,           # inspired from paper (Nb distribution part)
          'kwargs':{'gmax':200, 'pop':99, 'c1':1.494, 'c2':1.494, 'boxing':True}}, 
    'sampling_maxbox_40':{'alg':boxes.sampling,
                      'kwargs':{'inner_algorithm':boxes.maximal_box_sampling, 
                                'n_repeat':40, 'strategy':'small_first', 'maxbox_sampling':True,
                               'outer_boxing':True}},
    'sampling_maxbox_10':{'alg':boxes.sampling,
                      'kwargs':{'inner_algorithm':boxes.maximal_box_sampling, 
                                'n_repeat':10, 'strategy':'small_first', 'maxbox_sampling':True,
                                'outer_boxing':True}}    
}

algs_lb_sp={
    'fuzzy':{'alg':boxes.fuzzy,
            'kwargs':{'boxing':True}},
    'merge':{'alg':boxes.merge_algorithm,
             'kwargs':{'return_for_sa':False, 'boxing':True, 'measure_time':True}},
    'sa':{'alg':boxes.simulated_annealing,
         'kwargs':{'k1':5000, 'k2':5, 'k3':20, 'temp':0.6, 'cc':0.995}},
    
}

algs_rb_dd={
    'mcwr_0.75':{'alg':boxes.mcwr,
                 'kwargs':{'p':0.75,'boxing':True}},
    'mcwr_0.5':{'alg':boxes.mcwr,
                'kwargs':{'p':0.5,'boxing':True}},
    'mcwr_0.25':{'alg':boxes.mcwr,
                 'kwargs':{'p':0.25,'boxing':True}},
    'memb':{'alg':boxes.memb,
            'kwargs':{'boxing':True}},
    'random_sequential':{'alg':boxes.random_sequential,
                         'kwargs':{'boxing':True}},
    'remcc':{'alg':boxes.remcc,
             'kwargs':{'return_centres':False}},
    'sampling_rs_40':{'alg':boxes.sampling, # boxing goes to random sequential hopefully
                      'kwargs':{'inner_algorithm':boxes.random_sequential,
                               'n_repeat':40, 'strategy':'small_first', 'maxbox_sampling':False,
                                'outer_boxing':True,'boxing':False}}, 
    'sampling_rs_10':{'alg':boxes.sampling,
                      'kwargs':{'inner_algorithm':boxes.random_sequential, 
                                'n_repeat':10, 'strategy':'small_first', 'maxbox_sampling':False,
                                'outer_boxing':True,'boxing':False}},
}

In [4]:
resource.setrlimit(resource.RLIMIT_AS, (int(14e9),int(14e9))) # control memory usage, in bytes

In [8]:
output_dir='logs'
os.mkdir(output_dir)

n_bnch=15 # how many independent iterations shall be done?

In [9]:
for idx,net_ in enumerate(networks['name']):
    
    nw=boxes.network(boxes.load.read_max_connected_component(os.path.join(networks['path'][idx],
                                                                          networks['fname'][idx]),
                                                             networks['head'][idx]))
    d=nw.diameter
    
    
    # maybe log spacing would be sensible, but for the nets of our size: OK
    rb_vals=list(range(1,d,max(int(d/15),1))) # we sample at most ~ 15 possible rb vals
    lb_vals=[2*i for i in rb_vals] # we want to have same eqiuvalent sizes
    
    print('loaded '+networks['name'][idx]+' network')
    print('number of nodes %d, diameter: %d' %(nw.graph.number_of_nodes(),nw.diameter))
    
    # lb algorithms, distance dict
    
    boxes.boxing_(network=nw,
                  net=net_,
                  alg_dict=algs_lb_dd,
                  box_sizes=lb_vals,
                  path=os.path.join(output_dir,net_)+'/',
                  preprocessing='distance_dict',
                  benchmark_n=n_bnch
                  )
    
    # lb with shortest path matrix only
    
    boxes.boxing_(network=nw,
                  net=net_,
                  alg_dict=algs_lb_sp,
                  box_sizes=lb_vals,
                  path=os.path.join(output_dir,net_)+'/',
                  preprocessing='shortest_paths',
                  benchmark_n=n_bnch
                  )
    
    #rb (with all distance dict)
    
    boxes.boxing_(network=nw,
                  net=net_,
                  alg_dict=algs_rb_dd,
                  box_sizes=rb_vals,
                  path=os.path.join(output_dir,net_)+'/',
                  preprocessing='distance_dict',
                  benchmark_n=n_bnch
                  )
    del(nw)
    gc.collect()

loaded phd network
number of nodes 1025, diameter: 28


KeyboardInterrupt: 